# Quantitative Value Strategy

Investing in stocks that are cheapest relative to common measures of business value (like earnings or assets).

In [ ]:
# Import libraries

import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore
import xlsxwriter

In [ ]:
# Import list of stocks

stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks.drop([135,219, 467, 484]) # stocks giving error, might not be in list
print(stocks)

In [ ]:
# Import IEX Cloud API Token (Sandbox API)

from secret import IEX_CLOUD_API_TOKEN
token = IEX_CLOUD_API_TOKEN

In [ ]:
# Make first API Call
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={token}"
data = requests.get(api_url).json()
# data

## Execute Batch API Call and Build Dataframe.

In [ ]:
def divide_chunks(lst, n):     
    """Yield successive n sized chunks/batches from lst"""
    for i in range(0, len(lst), n):
        yield lst.loc[i:i + n]
        
symbol_groups = list(divide_chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of shares to buy']

In [ ]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    # Make Batch API Call 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={token}'
    response = requests.get(batch_api_call_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
#     print(data)

    for symbol in symbol_string.split(','):
        
        if symbol in data.keys():
            # Parse API Call
            price = data[symbol]['quote']['latestPrice']
            peRatio = data[symbol]['quote']['peRatio']

            # Add stocks to dataframe
            new_stock = pd.DataFrame([[symbol, price, peRatio, "N/A"]], columns = my_columns)
            final_df = pd.concat([final_df, new_stock])

## Remove glamour stocks.

In [ ]:
final_df.sort_values('Price-to-Earnings Ratio', inplace = True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df.drop('index', axis=1, inplace = True)

## Calculate number of shares to buy.

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
    except ValueError:
        print("Please enter an integer.")
        portfolio_size = input('Enter the value of your portfolio:')
        val = float(portfolio_size)

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(final_df.index)
final_df['Number of shares to buy'] = (position_size/final_df['Price']).apply(np.floor)
# final_df

## Building a better and more realistic Value Strategy.

Using a `composite` basket of valuation metrics to build robust quantitative value strategies.

We will filter for stocks with the lowest percentiles on the following metrics:
* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_columns)
# rv_df

In [ ]:
for symbol_string in symbol_strings[:1]:
    # Make Batch API Call 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={token}'
    response = requests.get(batch_api_call_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
    
    for symbol in symbol_string.split(','):
        
        if symbol in data.keys():
            # Parse API Call
            price = data[symbol]['quote']['latestPrice']
            peRatio = data[symbol]['stats']['peRatio']
            priceToBook = data[symbol]['stats']['priceToBook']
            priceToSales = data[symbol]['stats']['priceToSales']
            
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = enterprise_value/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN

            try:
                ev_to_gross_profit = enterprise_value/gross_profit
            except TypeError:
                ev_to_gross_profit = np.NaN


            # Add stocks to dataframe
            new_stock = pd.DataFrame([[
                symbol, 
                price, 
                "N/A", 
                peRatio,
                "N/A",
                priceToBook, 
                "N/A",
                priceToSales, 
                "N/A",
                ev_to_ebitda,
                "N/A",
                ev_to_gross_profit,
                "N/A",
                "N/A"
            ]], columns = hqm_columns)
            rv_df = pd.concat([rv_df, new_stock])

## Identify missing data in dataframe.

In [ ]:
rv_df[rv_dataframe.isnull().any(axis=1)]

In [ ]:
# replace NaN with average value
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace = True)

In [ ]:
rv_df[rv_dataframe.isnull().any(axis=1)]

## Calculating Value Percentiles.

In [ ]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_df.index:
    for metric in metrics.keys():
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_df[metric])

#Print the entire DataFrame    
rv_df

## Calculating RV Score.

In [ ]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_df

## Select 50 best Value Stocks.

In [ ]:
rv_df.sort_values(by = 'RV Score', inplace = True)
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)

## Calculate number of shares to buy.


In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(final_df.index)
rv_df['Number of shares to buy'] = (position_size/rv_df['Price']).apply(np.floor)
# rv_df

## Format Excel Output.

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [ ]:
# Set Formats

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
# Apply formats
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
# Save .xlsx file
writer.save()